# Import libraries

In [9]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from utils import confusion
from tqdm import tqdm

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Generate predictions in LOOCV

In [3]:
np.random.seed(42)

In [4]:
leave = LeaveOneOut()
logit_prediction_list = []
    
# instantiate the logistic regression classifier (gmhi2)
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

# generate logit predictions in LOOCV
GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
    cv=leave, verbose=2, n_jobs=-1
)

GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
logit_prediction_list.append(GMHI2_scores_cv)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 616 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 981 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1426 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1953 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 2560 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 3249 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 4869 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 5800 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 6813 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 7906 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 8069 out of 8069 |

In [5]:
def evaluate_performance(cutoff):
    balanced_accuracies = [balanced_accuracy_score(y[(abs(lo) >= cutoff).values], lo[(abs(lo) >= cutoff).values] > 0) for lo in logit_prediction_list]
    mean_acc = np.mean(balanced_accuracies)
    print("Mean acc:", mean_acc)
    
    mat_sum = np.zeros([2 , 2])

    for lo in logit_prediction_list:
        idx = (abs(lo) >= cutoff).values
        y_curr = y[idx]
        lo_curr = lo[idx]
        mat = confusion_matrix(y_curr, lo_curr > 0)
        mat_sum += mat
    tn, fp, fn, tp = mat_sum.ravel()
    df = pd.DataFrame(mat_sum, columns=["Predicted Nonhealthy", "Predicted Healthy"], index=["Actual Nonhealthy", "Actual healthy"])
    display(df)
    print("Total samples evaluated:", mat_sum.sum())
    print("Percentage of possible:", mat_sum.sum() * 100 / X.shape[0])

In [6]:
evaluate_performance(0)

Mean acc: 0.7906874882322742


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,2005.0,517.0
Actual healthy,1185.0,4362.0


Total samples evaluated: 8069.0
Percentage of possible: 100.0


In [7]:
evaluate_performance(0.1)

Mean acc: 0.8027134554727597


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,1936.0,465.0
Actual healthy,1067.0,4244.0


Total samples evaluated: 7712.0
Percentage of possible: 95.5756599330772


In [8]:
evaluate_performance(0.5)

Mean acc: 0.8495084012332856


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,1653.0,287.0
Actual healthy,676.0,3741.0


Total samples evaluated: 6357.0
Percentage of possible: 78.78299665386045


In [9]:
evaluate_performance(1)

Mean acc: 0.9035385477176245


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,1245.0,126.0
Actual healthy,337.0,2999.0


Total samples evaluated: 4707.0
Percentage of possible: 58.3343660924526


In [5]:
save_path = os.path.join(config.LOG_DIR, "LOOCV.csv")

In [13]:
logit_prediction_list[0].to_csv(save_path)

In [8]:
logit_predictions = pd.read_csv(save_path, index_col=[0, 1])
logit_predictions

GMHI2_cv
Study_ID  Sample Accession          
GMHI-10   SAMN03283239      1.830409
          SAMN03283266      0.085984
          SAMN03283281      1.098292
          SAMN03283294      1.752798
          SAMN03283288      1.660055
...                              ...
Yang_2020 SRR6456373       -1.050490
          SRR6456374       -0.974017
          SRR6456375       -0.048642
          SRR6456376        0.961760
          SRR6456377       -0.093069

[8069 rows x 1 columns]

In [12]:
display(confusion(logit_predictions, y, 0))
display(confusion(logit_predictions, y, 0.1))
display(confusion(logit_predictions, y, 0.5))
display(confusion(logit_predictions, y, 1))

,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,517.0,2005.0,0.795004
Actual healthy,4362.0,1185.0,0.786371


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,465.0,1936.0,0.806331
Actual healthy,4244.0,1067.0,0.799096


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,287.0,1653.0,0.852062
Actual healthy,3741.0,676.0,0.846955


,Predicted Healthy,Predicted Nonhealthy,Accuracy
Actual Nonhealthy,126.0,1245.0,0.908096
Actual healthy,2999.0,337.0,0.898981
